# Основи маніпулювання даними з `dplyr`

## 1. Інсталяція та огляд пакету `dplyr`

In [2]:
# install.packages("dplyr")

In [2]:
library(dplyr)

## Завантаження набору даних

In [7]:
# install.packages("gapminder")

In [3]:
library(gapminder)

In [4]:
head(gapminder)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530
Afghanistan,Asia,1962,31.997,10267083,853.1007
Afghanistan,Asia,1967,34.020,11537966,836.1971
Afghanistan,Asia,1972,36.088,13079460,739.9811
Afghanistan,Asia,1977,38.438,14880372,786.1134


In [13]:
?gapminder

gapminder {gapminder},R Documentation


In [5]:
str(gapminder)

tibble [1,704 x 6] (S3: tbl_df/tbl/data.frame)
 $ country  : Factor w/ 142 levels "Afghanistan",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ continent: Factor w/ 5 levels "Africa","Americas",..: 3 3 3 3 3 3 3 3 3 3 ...
 $ year     : int [1:1704] 1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 ...
 $ lifeExp  : num [1:1704] 28.8 30.3 32 34 36.1 ...
 $ pop      : int [1:1704] 8425333 9240934 10267083 11537966 13079460 14880372 12881816 13867957 16317921 22227415 ...
 $ gdpPercap: num [1:1704] 779 821 853 836 740 ...


In [6]:
glimpse(gapminder)

Rows: 1,704
Columns: 6
$ country   <fct> "Afghanistan", "Afghanistan", "Afghanistan", "Afghanistan", ~
$ continent <fct> Asia, Asia, Asia, Asia, Asia, Asia, Asia, Asia, Asia, Asia, ~
$ year      <int> 1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, ~
$ lifeExp   <dbl> 28.801, 30.332, 31.997, 34.020, 36.088, 38.438, 39.854, 40.8~
$ pop       <int> 8425333, 9240934, 10267083, 11537966, 13079460, 14880372, 12~
$ gdpPercap <dbl> 779.4453, 820.8530, 853.1007, 836.1971, 739.9811, 786.1134, ~


In [7]:
summary(gapminder)

        country        continent        year         lifeExp     
 Afghanistan:  12   Africa  :624   Min.   :1952   Min.   :23.60  
 Albania    :  12   Americas:300   1st Qu.:1966   1st Qu.:48.20  
 Algeria    :  12   Asia    :396   Median :1980   Median :60.71  
 Angola     :  12   Europe  :360   Mean   :1980   Mean   :59.47  
 Argentina  :  12   Oceania : 24   3rd Qu.:1993   3rd Qu.:70.85  
 Australia  :  12                  Max.   :2007   Max.   :82.60  
 (Other)    :1632                                                
      pop              gdpPercap       
 Min.   :6.001e+04   Min.   :   241.2  
 1st Qu.:2.794e+06   1st Qu.:  1202.1  
 Median :7.024e+06   Median :  3531.8  
 Mean   :2.960e+07   Mean   :  7215.3  
 3rd Qu.:1.959e+07   3rd Qu.:  9325.5  
 Max.   :1.319e+09   Max.   :113523.1  
                                       

## 3. `filter()`

In [8]:
canada <- filter(gapminder, country == "Canada")
canada

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Canada,Americas,1952,68.750,14785584,11367.16
Canada,Americas,1957,69.960,17010154,12489.95
Canada,Americas,1962,71.300,18985849,13462.49
Canada,Americas,1967,72.130,20819767,16076.59
Canada,Americas,1972,72.880,22284500,18970.57
Canada,Americas,1977,74.210,23796400,22090.88
Canada,Americas,1982,75.760,25201900,22898.79
Canada,Americas,1987,76.860,26549700,26626.52
Canada,Americas,1992,77.950,28523502,26342.88


In [9]:
filter(gapminder, lifeExp < 31)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530
Angola,Africa,1952,30.015,4232095,3520.6103
Gambia,Africa,1952,30.000,284320,485.2307
Rwanda,Africa,1992,23.599,7290203,737.0686
Sierra Leone,Africa,1952,30.331,2143249,879.7877


In [10]:
filter(gapminder, country == "Canada", year > 1990)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Canada,Americas,1992,77.950,28523502,26342.88
Canada,Americas,1997,78.610,30305843,28954.93
Canada,Americas,2002,79.770,31902268,33328.97
Canada,Americas,2007,80.653,33390141,36319.24


In [29]:
filter(gapminder, country %in% c("Austria", "Belgium"), year > 2000, gdpPercap > 31000)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Austria,Europe,2002,78.980,8148312,32417.61
Austria,Europe,2007,79.829,8199783,36126.49
Belgium,Europe,2007,79.441,10392226,33692.61


## 4. Оператор `pipe`

**`%>%`** #magritr

**`|>`** R 4.1

In [11]:
canada <- gapminder |> 
    filter(country == "Canada")    
canada

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Canada,Americas,1952,68.750,14785584,11367.16
Canada,Americas,1957,69.960,17010154,12489.95
Canada,Americas,1962,71.300,18985849,13462.49
Canada,Americas,1967,72.130,20819767,16076.59
Canada,Americas,1972,72.880,22284500,18970.57
Canada,Americas,1977,74.210,23796400,22090.88
Canada,Americas,1982,75.760,25201900,22898.79
Canada,Americas,1987,76.860,26549700,26626.52
Canada,Americas,1992,77.950,28523502,26342.88


In [35]:
set.seed(2025)

data <- data.frame(
    Gender = c("M", "M", "F", "F", "M", "F"),
    Age = sample(12:40, size = 6),
    IsActiveClient = c(T,F,F,F,T,T))
data

Gender,Age,IsActiveClient
<chr>,<int>,<lgl>
M,24,TRUE
M,23,FALSE
F,15,FALSE
F,37,FALSE
M,12,TRUE
F,34,TRUE


In [37]:
data |> select(Gender, Age)

Gender,Age
<chr>,<int>
M,24
M,23
F,15
F,37
M,12
F,34


In [39]:
data |>
    select(Gender, Age) |>
    filter(Age > 18) 

Gender,Age
<chr>,<int>
M,24
M,23
F,37
F,34


In [41]:
data |>
    select(Gender, Age) |>
    filter(Age > 18) |>
    group_by(Gender) |>
    summarise(MeanAge = mean(Age))

Gender,MeanAge
<chr>,<dbl>
F,35.5
M,23.5


In [43]:
head(gapminder)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530
Afghanistan,Asia,1962,31.997,10267083,853.1007
Afghanistan,Asia,1967,34.020,11537966,836.1971
Afghanistan,Asia,1972,36.088,13079460,739.9811
Afghanistan,Asia,1977,38.438,14880372,786.1134


In [46]:
gapminder |> head(3)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530
Afghanistan,Asia,1962,31.997,10267083,853.1007


## 5. `select`

In [50]:
gapminder |> 
    filter(country == "Canada") |>
    select(year, country, pop) |>
    head()

year,country,pop
<int>,<fct>,<int>
1952,Canada,14785584
1957,Canada,17010154
1962,Canada,18985849
1967,Canada,20819767
1972,Canada,22284500
1977,Canada,23796400


In [53]:
gapminder |> 
    filter(country == "Canada") |>
    select(-year, -country, -continent) |>
    head()

lifeExp,pop,gdpPercap
<dbl>,<int>,<dbl>
68.75,14785584,11367.16
69.96,17010154,12489.95
71.30,18985849,13462.49
72.13,20819767,16076.59
72.88,22284500,18970.57
74.21,23796400,22090.88


In [58]:
gapminder |>
    select(!where(is.numeric)) |>
    distinct() |>
    slice(1:5)

country,continent
<fct>,<fct>
Afghanistan,Asia
Albania,Europe
Algeria,Africa
Angola,Africa
Argentina,Americas


## 6. Random sampling

In [63]:
set.seed(10)
gapminder |> sample_n(5)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Equatorial Guinea,Africa,2002,49.348,495627,7703.4959
"Yemen, Rep.",Asia,1972,39.848,7407075,1265.0470
Sierra Leone,Africa,1997,39.897,4578212,574.6482
Cote d'Ivoire,Africa,1987,54.655,10761098,2156.9561
Switzerland,Europe,1987,77.410,6649942,30281.7046


In [65]:
set.seed(10)
gapminder |> sample_frac(0.01)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Equatorial Guinea,Africa,2002,49.348,495627,7703.4959
"Yemen, Rep.",Asia,1972,39.848,7407075,1265.0470
Sierra Leone,Africa,1997,39.897,4578212,574.6482
Cote d'Ivoire,Africa,1987,54.655,10761098,2156.9561
Switzerland,Europe,1987,77.410,6649942,30281.7046
Dominican Republic,Americas,1982,63.727,5968349,2861.0924
United Kingdom,Europe,2007,79.425,60776238,33203.2613
Swaziland,Africa,1997,54.289,1054486,3876.7685
Sudan,Africa,1982,50.338,20367053,1895.5441


## 7. `slice_`

In [71]:
gapminder |> slice_head(n = 6)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530
Afghanistan,Asia,1962,31.997,10267083,853.1007
Afghanistan,Asia,1967,34.020,11537966,836.1971
Afghanistan,Asia,1972,36.088,13079460,739.9811
Afghanistan,Asia,1977,38.438,14880372,786.1134


In [73]:
gapminder |> slice_tail(n = 6)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Zimbabwe,Africa,1982,60.363,7636524,788.8550
Zimbabwe,Africa,1987,62.351,9216418,706.1573
Zimbabwe,Africa,1992,60.377,10704340,693.4208
Zimbabwe,Africa,1997,46.809,11404948,792.4500
Zimbabwe,Africa,2002,39.989,11926563,672.0386
Zimbabwe,Africa,2007,43.487,12311143,469.7093


In [76]:
gapminder |>
    slice(-c(1:5, 7)) |>
    slice_head(n = 6)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Afghanistan,Asia,1977,38.438,14880372,786.1134
Afghanistan,Asia,1987,40.822,13867957,852.3959
Afghanistan,Asia,1992,41.674,16317921,649.3414
Afghanistan,Asia,1997,41.763,22227415,635.3414
Afghanistan,Asia,2002,42.129,25268405,726.7341
Afghanistan,Asia,2007,43.828,31889923,974.5803


In [78]:
gapminder |>
    slice_sample(n = 5)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Italy,Europe,1982,74.980,56535636,16537.483
Puerto Rico,Americas,1982,73.750,3279001,10330.989
Zambia,Africa,2002,39.193,10595811,1071.614
Singapore,Asia,1997,77.158,3802309,33519.477
Singapore,Asia,1977,70.795,2325300,11210.089


In [80]:
gapminder |>
    slice_min(lifeExp, n = 5)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Rwanda,Africa,1992,23.599,7290203,737.0686
Afghanistan,Asia,1952,28.801,8425333,779.4453
Gambia,Africa,1952,30.000,284320,485.2307
Angola,Africa,1952,30.015,4232095,3520.6103
Sierra Leone,Africa,1952,30.331,2143249,879.7877


In [82]:
gapminder |>
    slice_max(lifeExp, n = 5)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Japan,Asia,2007,82.603,127467972,31656.07
"Hong Kong, China",Asia,2007,82.208,6980412,39724.98
Japan,Asia,2002,82.000,127065841,28604.59
Iceland,Europe,2007,81.757,301931,36180.79
Switzerland,Europe,2007,81.701,7554661,37506.42


## 7. `arrange`

In [84]:
gapminder |>
    filter(year == 2007) |>
    top_n(10, lifeExp)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Australia,Oceania,2007,81.235,20434176,34435.37
Canada,Americas,2007,80.653,33390141,36319.24
France,Europe,2007,80.657,61083916,30470.02
"Hong Kong, China",Asia,2007,82.208,6980412,39724.98
Iceland,Europe,2007,81.757,301931,36180.79
Israel,Asia,2007,80.745,6426679,25523.28
Japan,Asia,2007,82.603,127467972,31656.07
Spain,Europe,2007,80.941,40448191,28821.06
Sweden,Europe,2007,80.884,9031088,33859.75


In [88]:
gapminder |>
    filter(year == 2007) |>
    arrange(desc(pop)) |>
    top_n(10, pop)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
China,Asia,2007,72.961,1318683096,4959.115
India,Asia,2007,64.698,1110396331,2452.210
United States,Americas,2007,78.242,301139947,42951.653
Indonesia,Asia,2007,70.650,223547000,3540.652
Brazil,Americas,2007,72.390,190010647,9065.801
Pakistan,Asia,2007,65.483,169270617,2605.948
Bangladesh,Asia,2007,64.062,150448339,1391.254
Nigeria,Africa,2007,46.859,135031164,2013.977
Japan,Asia,2007,82.603,127467972,31656.068


## 8. `mutate`

In [102]:
data <- gapminder |>
    filter(year == 2007) |>
    arrange(pop) 

In [103]:
data |> head()

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Sao Tome and Principe,Africa,2007,65.528,199579,1598.435
Iceland,Europe,2007,81.757,301931,36180.789
Djibouti,Africa,2007,54.791,496374,2082.482
Equatorial Guinea,Africa,2007,51.579,551201,12154.090
Montenegro,Europe,2007,74.543,684736,9253.896
Bahrain,Asia,2007,75.635,708573,29796.048


In [104]:
data <- data |>
    mutate(gdpTotal = pop * gdpPercap) 

In [95]:
data |> head()

country,continent,year,lifeExp,pop,gdpPercap,gdpTotal
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>,<dbl>
China,Asia,2007,72.961,1318683096,4959.115,6.539501e+12
India,Asia,2007,64.698,1110396331,2452.210,2.722925e+12
United States,Americas,2007,78.242,301139947,42951.653,1.293446e+13
Indonesia,Asia,2007,70.650,223547000,3540.652,7.915020e+11
Brazil,Americas,2007,72.390,190010647,9065.801,1.722599e+12
Pakistan,Asia,2007,65.483,169270617,2605.948,4.411104e+11


In [105]:
data |> transmute(gdpTotal = pop * gdpPercap) |> head()

gdpTotal
<dbl>
319014077
10924101861
1033689705
6699346424
6336475807
21112675360


In [106]:
data <- data |>
    mutate(gdpTotal = pop * gdpPercap,
           country = toupper(country),
           lifeExp = round(lifeExp))

In [108]:
data |> head()

country,continent,year,lifeExp,pop,gdpPercap,gdpTotal
<chr>,<fct>,<int>,<dbl>,<int>,<dbl>,<dbl>
SAO TOME AND PRINCIPE,Africa,2007,66,199579,1598.435,319014077
ICELAND,Europe,2007,82,301931,36180.789,10924101861
DJIBOUTI,Africa,2007,55,496374,2082.482,1033689705
EQUATORIAL GUINEA,Africa,2007,52,551201,12154.090,6699346424
MONTENEGRO,Europe,2007,75,684736,9253.896,6336475807
BAHRAIN,Asia,2007,76,708573,29796.048,21112675360


`Europe` -> `EU`

In [110]:
data <- data |>
    mutate(continent = as.character(continent),
          continent = ifelse(continent == "Europe", "EU", continent))

In [111]:
data |> head()

country,continent,year,lifeExp,pop,gdpPercap,gdpTotal
<chr>,<chr>,<int>,<dbl>,<int>,<dbl>,<dbl>
SAO TOME AND PRINCIPE,Africa,2007,66,199579,1598.435,319014077
ICELAND,EU,2007,82,301931,36180.789,10924101861
DJIBOUTI,Africa,2007,55,496374,2082.482,1033689705
EQUATORIAL GUINEA,Africa,2007,52,551201,12154.090,6699346424
MONTENEGRO,EU,2007,75,684736,9253.896,6336475807
BAHRAIN,Asia,2007,76,708573,29796.048,21112675360


## 9. `rename`

In [112]:
data <- data |>
    rename(population = pop)

#rename_if
#rename_at

In [113]:
data |> head()

country,continent,year,lifeExp,population,gdpPercap,gdpTotal
<chr>,<chr>,<int>,<dbl>,<int>,<dbl>,<dbl>
SAO TOME AND PRINCIPE,Africa,2007,66,199579,1598.435,319014077
ICELAND,EU,2007,82,301931,36180.789,10924101861
DJIBOUTI,Africa,2007,55,496374,2082.482,1033689705
EQUATORIAL GUINEA,Africa,2007,52,551201,12154.090,6699346424
MONTENEGRO,EU,2007,75,684736,9253.896,6336475807
BAHRAIN,Asia,2007,76,708573,29796.048,21112675360


## 10. `group_by` + `summarise`

In [115]:
data <- gapminder |>
    filter(year == 2007) |>
    group_by(continent) |>
    summarise(
        lifeExp2007 = mean(lifeExp),
        countriesCount = n()
    )

In [116]:
data |> head()

continent,lifeExp2007,countriesCount
<fct>,<dbl>,<int>
Africa,54.80604,52
Americas,73.60812,25
Asia,70.72848,33
Europe,77.64860,30
Oceania,80.71950,2


In [117]:
data <- gapminder |>
    filter(year == 2007) |>
    group_by(continent) |>
    summarise(
        pop = sum(pop)
    )

In [121]:
data

continent,pop
<fct>,<dbl>
Africa,929539692
Americas,898871184
Asia,3811953827
Europe,586098529
Oceania,24549947


In [122]:
sum(data$pop)

[1] 6251013179

In [123]:
data <- data |>
    mutate(pop_perc = round(100*pop/sum(data$pop), 2))
    

In [125]:
data

continent,pop,pop_perc
<fct>,<dbl>,<dbl>
Africa,929539692,14.87
Americas,898871184,14.38
Asia,3811953827,60.98
Europe,586098529,9.38
Oceania,24549947,0.39


In [128]:
sum(data$pop_perc)

[1] 100

## 11. `bind_`

In [12]:
data1 <- gapminder |>
    filter(year == 2007) |>
    group_by(continent) |>
    summarise(
        lifeExp2007 = mean(lifeExp),
        countriesCount = n()
    )
data1

continent,lifeExp2007,countriesCount
<fct>,<dbl>,<int>
Africa,54.80604,52
Americas,73.60812,25
Asia,70.72848,33
Europe,77.64860,30
Oceania,80.71950,2


In [14]:
data2 <- gapminder |>
    filter(year == 2007) |>
    group_by(continent) |>
    summarise(
        pop = sum(pop)
    ) 
data2 <- data2 |>
    mutate(pop_perc = round(100*pop/sum(data2$pop), 2)) |>
    arrange(pop)
data2

continent,pop,pop_perc
<fct>,<dbl>,<dbl>
Oceania,24549947,0.39
Europe,586098529,9.38
Americas,898871184,14.38
Africa,929539692,14.87
Asia,3811953827,60.98


In [15]:
data_all <- data1 |> 
    bind_cols(data2) #|>
    #select(-4) |>
    #rename(continent = `continent...1`)
data_all

New names:
* `continent` -> `continent...1`
* `continent` -> `continent...4`


continent...1,lifeExp2007,countriesCount,continent...4,pop,pop_perc
<fct>,<dbl>,<int>,<fct>,<dbl>,<dbl>
Africa,54.80604,52,Oceania,24549947,0.39
Americas,73.60812,25,Europe,586098529,9.38
Asia,70.72848,33,Americas,898871184,14.38
Europe,77.64860,30,Africa,929539692,14.87
Oceania,80.71950,2,Asia,3811953827,60.98


In [137]:
data1_2002 <- gapminder |>
    filter(year == 2002) |>
    group_by(continent) |>
    summarise(
        lifeExp2007 = mean(lifeExp),
        countriesCount = n()
    )
data1_2002

continent,lifeExp2007,countriesCount
<fct>,<dbl>,<int>
Africa,53.32523,52
Americas,72.42204,25
Asia,69.23388,33
Europe,76.70060,30
Oceania,79.74000,2


In [139]:
data <- data1 |> bind_rows(data1_2002)

## `JOIN`

In [19]:
data1 <- gapminder |>
    filter(year == 2007) |>
    group_by(continent) |>
    summarise(
        lifeExp2007 = mean(lifeExp),
        countriesCount = n()
    )
data1

continent,lifeExp2007,countriesCount
<fct>,<dbl>,<int>
Africa,53.32523,52
Americas,72.42204,25
Asia,69.23388,33
Europe,76.70060,30
Oceania,79.74000,2


In [21]:
data2 <- gapminder |>
    filter(year == 2007) |>
    group_by(continent) |>
    summarise(
        totalPop = sum(pop)
    ) |>
    arrange(totalPop)
data2

continent,totalPop
<fct>,<dbl>
Oceania,24549947
Europe,586098529
Americas,898871184
Africa,929539692
Asia,3811953827


In [23]:
all_data <- data1 |>
    left_join(data2, by = "continent")
all_data

continent,lifeExp2007,countriesCount,totalPop
<fct>,<dbl>,<int>,<dbl>
Africa,54.80604,52,929539692
Americas,73.60812,25,898871184
Asia,70.72848,33,3811953827
Europe,77.64860,30,586098529
Oceania,80.71950,2,24549947


In [26]:
df_1 <- data.frame(Letter = c("A", "B", "C", "D", "E"),
                      Value = c(1:5))

df_2 <- data.frame(Letter = c("A", "B", "C", "D", "F"),
                      Value = c(12, 7, 4, 1, 5))
df_1
df_2

Letter,Value
<chr>,<int>
A,1
B,2
C,3
D,4
E,5


Letter,Value
<chr>,<dbl>
A,12
B,7
C,4
D,1
F,5


In [28]:
df_1 |>
    left_join(df_2, by = "Letter")

Letter,Value.x,Value.y
<chr>,<int>,<dbl>
A,1,12
B,2,7
C,3,4
D,4,1
E,5,NA


In [30]:
df_1 |>
    right_join(df_2, by = "Letter")

Letter,Value.x,Value.y
<chr>,<int>,<dbl>
A,1,12
B,2,7
C,3,4
D,4,1
F,NA,5


In [32]:
df_1 |>
    inner_join(df_2, by = "Letter")

Letter,Value.x,Value.y
<chr>,<int>,<dbl>
A,1,12
B,2,7
C,3,4
D,4,1


In [34]:
df_1 |>
    full_join(df_2, by = "Letter")

Letter,Value.x,Value.y
<chr>,<int>,<dbl>
A,1,12
B,2,7
C,3,4
D,4,1
E,5,NA
F,NA,5


In [38]:
#install.packages("randomNames")
library(randomNames)

In [41]:
randomNames(4)

[1] "Xiong, Carly"        "el-Mahmoud, Humaina" "Ortega-Aragon, John"
[4] "Gipson, Celina"

In [46]:
not_good_data <- data.frame(
    Name = randomNames(5),
    q1_2023 = sample(1000:10000, size = 5),
    q2_2023 = sample(1000:10000, size = 5),
    q3_2023 = sample(1000:10000, size = 5),
    q4_2023 = sample(1000:10000, size = 5)
)

not_good_data

Name,q1_2023,q2_2023,q3_2023,q4_2023
<chr>,<int>,<int>,<int>,<int>
"Le, I'Kaika",3592,2203,9822,3232
"Miller, Tyrell",4384,7200,9556,8372
"Alvarez-Gonzalez, Miranda",1065,5219,6078,2552
"Johnson, Cynthia",9025,4335,9979,5355
"Hessler, Lane",7121,1474,1908,3562


In [47]:
library(tidyr)

In [54]:
better_data <- not_good_data |>
    gather(quater, salary, 2:5)
better_data

Name,quater,salary
<chr>,<chr>,<int>
"Le, I'Kaika",q1_2023,3592
"Miller, Tyrell",q1_2023,4384
"Alvarez-Gonzalez, Miranda",q1_2023,1065
"Johnson, Cynthia",q1_2023,9025
"Hessler, Lane",q1_2023,7121
"Le, I'Kaika",q2_2023,2203
"Miller, Tyrell",q2_2023,7200
"Alvarez-Gonzalez, Miranda",q2_2023,5219
"Johnson, Cynthia",q2_2023,4335


In [65]:
best_data <- better_data |>
    separate(quater, c("quater", "year"), sep="_") |>
    mutate(
        year = as.numeric(year),
        quater = substr(quater, 2, 2),
        quater = as.numeric(quater))

best_data


Name,quater,year,salary
<chr>,<dbl>,<dbl>,<int>
"Le, I'Kaika",1,2023,3592
"Miller, Tyrell",1,2023,4384
"Alvarez-Gonzalez, Miranda",1,2023,1065
"Johnson, Cynthia",1,2023,9025
"Hessler, Lane",1,2023,7121
"Le, I'Kaika",2,2023,2203
"Miller, Tyrell",2,2023,7200
"Alvarez-Gonzalez, Miranda",2,2023,5219
"Johnson, Cynthia",2,2023,4335


In [69]:
united_data <- best_data |>
    unite(QtYear, quater, year, sep = "_") |>
    mutate(QtYear = paste0("q", QtYear))
united_data

Name,QtYear,salary
<chr>,<chr>,<int>
"Le, I'Kaika",q1_2023,3592
"Miller, Tyrell",q1_2023,4384
"Alvarez-Gonzalez, Miranda",q1_2023,1065
"Johnson, Cynthia",q1_2023,9025
"Hessler, Lane",q1_2023,7121
"Le, I'Kaika",q2_2023,2203
"Miller, Tyrell",q2_2023,7200
"Alvarez-Gonzalez, Miranda",q2_2023,5219
"Johnson, Cynthia",q2_2023,4335


In [71]:
final_data <- united_data |>
    spread(QtYear, salary)
final_data

Name,q1_2023,q2_2023,q3_2023,q4_2023
<chr>,<int>,<int>,<int>,<int>
"Alvarez-Gonzalez, Miranda",1065,5219,6078,2552
"Hessler, Lane",7121,1474,1908,3562
"Johnson, Cynthia",9025,4335,9979,5355
"Le, I'Kaika",3592,2203,9822,3232
"Miller, Tyrell",4384,7200,9556,8372
